# LABORATORIO DATOS 2

Debe construir un grafo simple, no dirigido y ponderado que modele las rutas de
transporte aéreo utilizando las coordenadas geográficas de varios aeropuertos.
Para ello, debe investigar sobre como obtener la distancia entre dos coordenadas
geográficas ya que este valor será el peso de la arista que conecte a dos
aeropuertos.
Los datos con los que debe construir el grafo se encuentran en el dataset
flights_final.csv que contiene la información relevante de los vuelos entre diversos
aeropuertos del mundo. Este dataset cuenta con 66930 registros y contiene los
siguientes atributos:


• Source Airport Code: Código del aeropuerto origen.
• Source Airport Name: Nombre del aeropuerto origen.
• Source Airport City: Ciudad del aeropuerto origen.
• Source Airport Country: País del aeropuerto origen.
• Source Airport Latitude: Latitud geográfica del aeropuerto origen.
• Source Airport Longitude: Longitud geográfica del aeropuerto origen.
• Destination Airport Code: Código del aeropuerto destino.
• Destination Airport Name: Nombre del aeropuerto destino.
• Destination Airport City: Ciudad del aeropuerto destino.
• Destination Airport Country: País del aeropuerto destino.
• Destination Airport Latitude: Latitud geográfica del aeropuerto destino.
• Destination Airport Longitude: Longitud geográfica del aeropuerto destino.


Utilizando las coordenadas geográficas de cada aeropuerto se debe mostrar un
mapa con la geolocalización de cada uno de ellos.
Sobre el grafo generado, el usuario debe poder realizar (desde la consola o desde
la interfaz gráfica) las siguientes acciones:


• Dado un primer vértice por el código del aeropuerto o seleccionado mediante
la interfaz gráfica:


o Mostrar la información correspondiente al aeropuerto (código, nombre, ciudad, país, latitud y longitud). o Mostrar la información (código, nombre, ciudad, país, latitud y longitud) de los 10 aeropuertos cuyos caminos mínimos desde el vértice dado sean los más largos. Adicionalmente, se debe mostrar la distancia de los caminos.
• Dado un segundo vértice por el código del aeropuerto o seleccionado mediante la interfaz gráfica:
- Mostrar el camino mínimo entre el primer y el segundo vértice sobre el mapa de la interfaz gráfica, pasando por cada uno de los vértices intermedios del camino. Para cada vértice intermedio se debe mostrar la información del aeropuerto (código, nombre, ciudad, país, latitud y longitud).

In [1]:
!pip install graphviz==0.20.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 kB 8.0 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 23.3
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip install networkx==3.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 77.2 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 23.3
[notice] To update, run: pip install --upgrade pip


In [3]:
!pip install pygraphviz==1.11

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.8/120.8 kB 17.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [55 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build/lib.linux-x86_64-3.9
      creating build/lib.linux-x86_64-3.9/pygraphviz
      copying pygraphviz/scraper.py -> build/lib.linux-x86_64-3.9/pygraphviz
      copying pygraphviz/graphviz.py -> build/lib.linux-x86_64-3.9/pygraphviz
      copying pygraphviz/testing.py -> build/lib.linux-x86_64-3.9/pygraphviz
      copying pygraphviz/agraph.py -> build/lib.linux-x86_64-3.9/pygraphviz
      copying pygraphviz/__init__.py -> build/lib.linux-x86_64-3.9/pygraphviz
      creating build/lib.linux-x86_64-3.9/pygraphviz/tests
      copying pygraphviz/tests/test_subgraph.py -> build/lib.linux-x86_64-3.9/pygraphviz/tests
      copy

In [4]:
!pip install pygraphviz==1.11

  Using cached pygraphviz-1.11.zip (120 kB)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [55 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build/lib.linux-x86_64-3.9
      creating build/lib.linux-x86_64-3.9/pygraphviz
      copying pygraphviz/scraper.py -> build/lib.linux-x86_64-3.9/pygraphviz
      copying pygraphviz/graphviz.py -> build/lib.linux-x86_64-3.9/pygraphviz
      copying pygraphviz/testing.py -> build/lib.linux-x86_64-3.9/pygraphviz
      copying pygraphviz/agraph.py -> build/lib.linux-x86_64-3.9/pygraphviz
      copying pygraphviz/__init__.py -> build/lib.linux-x86_64-3.9/pygraphviz
      creating build/lib.linux-x86_64-3.9/pygraphviz/tests
      copying pygraphviz/tests/test_subgraph.py -> build/lib.linux-x86_64-3.9/pygraphviz/tests
      copying pygraphviz/tests/test_unicode.py 

In [5]:
import pandas as pd
import numpy as np
import graphviz as gv
from typing import Any, List, Optional, Tuple
from pprint import pprint
from queue import Queue
import networkx as nx
import matplotlib.pyplot as plt
import heapq
import plotly.graph_objects as go
import plotly.express as px
import geopandas as gpd
# import pygraphviz as pgv

In [6]:
df = pd.read_csv("flights.csv")
df.index = range(1, len(df) +1) #Para inciar el conteo del índice en 1 en lugar de 0
df.head()

,Source Airport Code,Source Airport Name,Source Airport City,Source Airport Country,Source Airport Latitude,Source Airport Longitude,Destination Airport Code,Destination Airport Name,Destination Airport City,Destination Airport Country,Destination Airport Latitude,Destination Airport Longitude
1,COK,Cochin International Airport,Kochi,India,10.152000,76.401901,SHJ,Sharjah International Airport,Sharjah,United Arab Emirates,25.328600,55.517200
2,GIG,Rio Galeão – Tom Jobim International Airport,Rio De Janeiro,Brazil,-22.809999,-43.250557,BSB,Presidente Juscelino Kubistschek International...,Brasilia,Brazil,-15.869167,-47.920834
3,TSF,Treviso-Sant'Angelo Airport,Treviso,Italy,45.648399,12.194400,BRI,Bari Karol Wojtyła Airport,Bari,Italy,41.138901,16.760599
4,CDG,Charles de Gaulle International Airport,Paris,France,49.012798,2.550000,FLR,Peretola Airport,Florence,Italy,43.810001,11.205100
5,CGH,Congonhas Airport,Sao Paulo,Brazil,-23.626110,-46.656387,CGB,Marechal Rondon Airport,Cuiaba,Brazil,-15.652900,-56.116699


## Distancia entre aeropuertos

### Distancia de HAVERSINE

La distancia calculada es una distancia en el espacio tridimensional (esférica) y se expresa generalmente en unidades como kilómetros o millas.

La fórmula Haversine toma en cuenta la curvatura de la Tierra y es especialmente útil para calcular distancias geodésicas en problemas relacionados con la navegación, geolocalización, y otras aplicaciones que involucran coordenadas geográficas.

La fórmula de Haversine se ve de la siguiente manera:

a = sin²(Δlat/2) + cos(lat1) * cos(lat2) * sin²(Δlon/2)
c = 2 * atan2(√a, √(1-a))
d = R * c

In [7]:
"""Las variables de longitud y de latitud del aeropuerto inicial son:
1. Source Airport Latitude
2. Source Airport Longitude

Las variables de longitud y latitud del aeropuerto final son:
1. Destination Airport Latitude
2. Destination Airport Longitude"""

'Las variables de longitud y de latitud del aeropuerto inicial son:\n1. Source Airport Latitude\n2. Source Airport Longitude\n\nLas variables de longitud y latitud del aeropuerto final son:\n1. Destination Airport Latitude\n2. Destination Airport Longitude'

In [8]:
radio_tierra = 6371.0

# Convierte las coordenadas de grados a radianes
df['Latitud_salida_rad'] = np.radians(df['Source Airport Latitude'])
df['Longitud_salida_rad'] = np.radians(df['Source Airport Longitude'])
df['Latitud_llegada_rad'] = np.radians(df['Destination Airport Latitude'])
df['Longitud_llegada_rad'] = np.radians(df['Destination Airport Longitude'])

# Calcula las diferencias de latitud y longitud
df['dLat'] = df['Latitud_llegada_rad'] - df['Latitud_salida_rad']
df['dLon'] = df['Longitud_llegada_rad'] - df['Longitud_salida_rad']

# Calcula la distancia utilizando la fórmula de Haversine
df['distancia'] = 2 * np.arcsin(np.sqrt(
    np.sin(df['dLat'] / 2)**2 +
    np.cos(df['Latitud_salida_rad']) * np.cos(df['Latitud_llegada_rad']) *
    np.sin(df['dLon'] / 2)**2
)) * radio_tierra

In [9]:
df.head()

,Source Airport Code,Source Airport Name,Source Airport City,Source Airport Country,Source Airport Latitude,Source Airport Longitude,Destination Airport Code,Destination Airport Name,Destination Airport City,Destination Airport Country,Destination Airport Latitude,Destination Airport Longitude,Latitud_salida_rad,Longitud_salida_rad,Latitud_llegada_rad,Longitud_llegada_rad,dLat,dLon,distancia
1,COK,Cochin International Airport,Kochi,India,10.152000,76.401901,SHJ,Sharjah International Airport,Sharjah,United Arab Emirates,25.328600,55.517200,0.177186,1.333465,0.442067,0.968958,0.264882,-0.364507,2774.426364
2,GIG,Rio Galeão – Tom Jobim International Airport,Rio De Janeiro,Brazil,-22.809999,-43.250557,BSB,Presidente Juscelino Kubistschek International...,Brasilia,Brazil,-15.869167,-47.920834,-0.398110,-0.754865,-0.276969,-0.836376,0.121140,-0.081512,913.971019
3,TSF,Treviso-Sant'Angelo Airport,Treviso,Italy,45.648399,12.194400,BRI,Bari Karol Wojtyła Airport,Bari,Italy,41.138901,16.760599,0.796715,0.212832,0.718009,0.292528,-0.078706,0.079695,622.306540
4,CDG,Charles de Gaulle International Airport,Paris,France,49.012798,2.550000,FLR,Peretola Airport,Florence,Italy,43.810001,11.205100,0.855435,0.044506,0.764629,0.195566,-0.090806,0.151060,879.346159
5,CGH,Congonhas Airport,Sao Paulo,Brazil,-23.626110,-46.656387,CGB,Marechal Rondon Airport,Cuiaba,Brazil,-15.652900,-56.116699,-0.412353,-0.814308,-0.273195,-0.979421,0.139159,-0.165114,1328.590433


# Crear el grafo

In [33]:
class grafo:
    def __init__(self):
        self.graf = {}

    def add_edge(self, source, destination, distance):
        if source not in self.graf:
            self.graf[source] = {}
        if destination not in self.graf:
            self.graf[destination] = {}

        self.graf[source][destination] = distance
        self.graf[destination][source] = distance

    def __str__(self):
        result = ""
        for source in self.graf:
            for destination, distance in self.graf[source].items():
                result += f"({source} -- {destination} : {distance} km)\n"
        return result

    def dijkstra(self, source_airport, destination_airport):
        if source_airport not in self.graf or destination_airport not in self.graf:
            return None  # Al menos uno de los aeropuertos no existe en el grafo

        distances = {vertex: float('inf') for vertex in self.graf}
        distances[source_airport] = 0

        previous_vertices = {}  # Para mantener un seguimiento de los vértices anteriores en el camino mínimo
        priority_queue = [(0, source_airport)]

        while priority_queue:
            current_distance, current_vertex = heapq.heappop(priority_queue)

            if current_vertex == destination_airport:
                break

            if current_distance > distances[current_vertex]:
                continue

            for neighbor, weight in self.graf[current_vertex].items():
                distance = current_distance + weight
                if distance < distances[neighbor]:
                    distances[neighbor] = distance
                    previous_vertices[neighbor] = current_vertex
                    heapq.heappush(priority_queue, (distance, neighbor))

        if distances[destination_airport] == float('inf'):
            return None

        path = []
        current = destination_airport
        while current != source_airport:
            path.append(current)
            current = previous_vertices[current]
        path.append(source_airport)

        path.reverse()

        # Calcular y mostrar las distancias entre aeropuertos en el camino mínimo
        distances_between_airports = []
        for i in range(len(path) - 1):
            current_airport = path[i]
            next_airport = path[i + 1]
            distance_between_airports = self.graf[current_airport][next_airport]
            distances_between_airports.append(distance_between_airports)

        for i, airport in enumerate(path[:-1]):
            print(f"Aeropuerto: {airport} -> Aeropuerto: {path[i + 1]}, Distancia: {distances_between_airports[i]} km")

        return path

In [11]:
# Crear una instancia de la clase Graph
flight_graph = grafo()


# Iterar a través de las filas del DataFrame y agregar aristas al grafo
for index, row in df.iterrows():
    source_airport = row['Source Airport Code']
    destination_airport = row['Destination Airport Code']
    distance = row['distancia']
    flight_graph.add_edge(source_airport, destination_airport, distance)

# Imprimir el grafo
print(flight_graph)

342.608321915502 km)
(JAX -- HOU : 1310.4660821722205 km)
(JAX -- IAH : 1312.0444373437995 km)
(DBV -- HAM : 1373.7165296202331 km)
(DBV -- BFS : 2227.7800340640497 km)
(DBV -- NCL : 2001.992919487193 km)
(DBV -- OSL : 2013.0361241733065 km)
(DBV -- DUS : 1302.6377197250554 km)
(DBV -- FRA : 1114.139196497724 km)
(DBV -- LGW : 1690.0700156587732 km)
(DBV -- ORY : 1408.5733945572838 km)
(DBV -- CGN : 1251.173032820659 km)
(DBV -- ARN : 1900.5270549960171 km)
(DBV -- SPU : 193.4939840159091 km)
(DBV -- MUC : 818.1843073122297 km)
(DBV -- HEL : 2025.6724553033198 km)
(DBV -- EDI : 2146.9803906389434 km)
(DBV -- EMA : 1848.4619994610762 km)
(DBV -- STR : 978.1478313015195 km)
(DBV -- BGO : 2160.2111122457472 km)
(DBV -- ATH : 704.3287667229868 km)
(DBV -- ZRH : 937.4632627573527 km)
(DBV -- GVA : 1047.5440622096696 km)
(DBV -- LUX : 1214.795670011648 km)
(DBV -- AMS : 1480.697814347311 km)
(DBV -- BHX : 1852.5923916941242 km)
(DBV -- MAN : 1932.333689700376 km)
(DBV -- LIL : 1457.793617388

# Camino mínimo

In [34]:
flight_graph = grafo()
for _, row in df.iterrows():
    source_airport = row['Source Airport Code'].strip().upper()
    destination_airport = row['Destination Airport Code'].strip().upper()
    distance = row['distancia']
    flight_graph.add_edge(source_airport, destination_airport, distance)

source_airport = input("Ingrese el código del aeropuerto de origen: ").strip().upper()
destination_airport = input("Ingrese el código del aeropuerto de destino: ").strip().upper()

camino_minimo = flight_graph.dijkstra(source_airport, destination_airport)

if camino_minimo:
    print("Camino mínimo:")
    print(" -> ".join(camino_minimo))
else:
    print("No se encontró un camino entre los aeropuertos especificados.")

Aeropuerto: TLJ -> Aeropuerto: MCG, Distancia: 19.87238494346707 km
Aeropuerto: MCG -> Aeropuerto: ANC, Distancia: 352.68052518116906 km
Aeropuerto: ANC -> Aeropuerto: ORD, Distancia: 4568.3772725268855 km
Aeropuerto: ORD -> Aeropuerto: SXM, Distancia: 3548.9225165573994 km
Aeropuerto: SXM -> Aeropuerto: PTP, Distancia: 258.961476306407 km
Aeropuerto: PTP -> Aeropuerto: FDF, Distancia: 194.6039526191359 km
Aeropuerto: FDF -> Aeropuerto: CAY, Distancia: 1440.6321083140847 km
Aeropuerto: CAY -> Aeropuerto: BEL, Distancia: 813.2162000825944 km
Camino mínimo:
TLJ -> MCG -> ANC -> ORD -> SXM -> PTP -> FDF -> CAY -> BEL


# Mostrar mapa de vuelos

In [13]:
'''
    Enviar búsqueda como diccionario
    Ingresar la categoría de entre las siguientes

    Source Airport Code,
    Source Airport Name,
    Source Airport City,
    Source Airport Country,
    Source Airport Latitude,
    Source Airport Longitude,
    Destination Airport Code,
    Destination Airport Name,
    Destination Airport City,
    'Destination Airport Country',
    'Destination Airport Latitude',
    'Destination Airport Longitude'

    E ingresar una lista con los criterios de búsqueda
    Ejemplo:

    {'Source Airport Code': ['IND']}
'''
def searchNodes (search:dict, df:pd.DataFrame) -> pd.DataFrame:

  flights = pd.DataFrame({col: [] for col in df.keys()})

  #Si no hay requisitos se devuelve el mismo dataframe
  if (len(search) == 0):
    return df

  for i in df.index:  #Recorre todo el dataframe y filtra aquellos que cumplan con el requisito
    if (all([any([df[col][i] == crit for crit in search[col]]) for col in search.keys()])):
      flight = pd.DataFrame({col: df[col][i] for col in df.keys()}, index=[len(flights)+1])
      flights = pd.concat([flights, flight])

  return flights

In [14]:
#Sirve para obtener los nodos (aeropuertos) de un dataframe de vuelos
def getNodesfromFlights (df:pd.DataFrame) -> pd.DataFrame:
  cols = [col for col in df.keys()]
  nodes_names_set = set()
  nodes_set = []

  #Busca los nodos no repetidos tanto de origen como destino
  for i in df.index:
    if (df['Source Airport Name'][i] not in nodes_names_set):
      nodes_names_set.add(df['Source Airport Name'][i])
      nodes_set.append([df[col][i] for col in cols[:6]])

    if (df['Destination Airport Name'][i] not in nodes_names_set):
      nodes_names_set.add(df['Destination Airport Name'][i])
      nodes_set.append([df[col][i] for col in cols[6:12]])


  cols = ['Airport Code', 'Airport Name',
          'Airport City', 'Airport Country',
          'Airport Latitude', 'Airport Longitude']

  #Converte los nodos a un nuevo dataframe
  nodes = pd.DataFrame({col: [] for col in cols})
  for node_set in nodes_set:
    node = pd.DataFrame({cols[i]: node_set[i] for i in range(len(node_set))}, index=[len(nodes)+1])
    nodes = pd.concat([nodes, node])

  return nodes

In [15]:
def graphNodes (search:dict, df:pd.DataFrame, drawlines: bool) -> pd.DataFrame:
  #Token de cuenta de mapbox para ejecutar el mapa
  mapbox_access_token = 'pk.eyJ1Ijoic2ViYXN0aWFubWFsZG9uYWRvMTk0NSIsImEiOiJjbGluYnRobHkwbDQyM2xwOGc4aGN5ZnpvIn0.Jal1X7da0VhVK8gkKrWBng'

  flights = searchNodes(search, df) #Se buscan los vuelos solicitados
  nodes = getNodesfromFlights(flights)  #Se consiguen los aeropuertos

  lon = [nodes['Airport Longitude'][i] for i in nodes.index]
  lat = [nodes['Airport Latitude'][i] for i in nodes.index]

  fig = go.Figure()

  #Si se solicita dibujar lineas : [True] | [False]
  if (drawlines):
    for i in flights.index:
      fig.add_trace(go.Scattermapbox(
      mode = "lines",
      lon = [flights['Source Airport Longitude'][i], flights['Destination Airport Longitude'][i]],
      lat = [flights['Source Airport Latitude'][i], flights['Destination Airport Latitude'][i]],
      hoverinfo='skip',
      marker = {'size': 1}))

  #Dibujo de los nodos
  fig.add_trace(go.Scattermapbox(
    mode = "markers+text",
    lon = lon, lat = lat,
    marker = {'size': 20, 'symbol': ["airport" for i in range(len(nodes))]},
    text = [nodes['Airport Name'][i] for i in nodes.index],textposition = "bottom right"))

  fig.update_layout(
      mapbox = {
          'accesstoken': mapbox_access_token,
          'style': "outdoors", 'zoom': 0.7},
      showlegend = False)

  fig.show(renderer="colab")

In [16]:
graphNodes({}, df, False)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=486643e5-6001-41e5-8c0f-ecb82d6b19ba' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>